# Set up

In [ ]:
params1 = list(
  counts   = paste("../data/scRNA_counts", "GSE184357_counts_frozen_postFilter.txt", sep = '/'),
  meta     = paste("../data/metadata", "GSE184357_metadata_frozen_postFilter.txt", sep='/'),
  out_name = "GSE184357_frozen",
  species  = "h_sapiens"
)
output_file1 = here::here("output/", glue::glue("preprocessing_GSE184357_frozen_", Sys.Date(), ".html"))

In [ ]:
params2 = list(
  counts   = paste("../data/scRNA_counts", "GSE184357_counts_fresh_postFilter.txt", sep = '/'),
  meta     = paste("../data/metadata", "GSE184357_metadata_fresh_postFilter.txt", sep='/'),
  out_name = "GSE184357_fresh",
  species  = "h_sapiens"
)
output_file2 = here::here("output/", glue::glue("preprocessing_GSE184357_fresh_", Sys.Date(), ".html"))

In [ ]:
params = list(
  out_name = "GSE184357_fresh&frozen",
  species  = "h_sapiens"
)
output_file3 = here::here("output/", glue::glue("preprocessing_GSE184357_fresh&frozen_", Sys.Date(), ".html"))

In [ ]:
# set up
knitr::opts_chunk$set(
  eval       = TRUE,
  echo       = TRUE,
	error      = FALSE,
	fig.align  = "center",
	message    = TRUE,
	warning    = FALSE,
	autodep    = TRUE,
	cache      = FALSE,
	cache.lazy = FALSE,
	results    = "markup",
  fig.path   = here::here(paste("Figures/", params$out_name, sep="/")),
  fig.keep   = "all",
  dev        = "png"
)

In [ ]:
set.seed(42)


In [ ]:
# out_dir
folder_path <- here::here(paste("figures", params$out_name, sep = "/"))
if (!dir.exists(folder_path)) {
  dir.create(folder_path)
} 

In [ ]:
# Load libraries
library(here)
library(Seurat)
library(dplyr)
library(stringr)
library(ggplot2)
library(glue)
library(harmony)
library(magrittr)
library(DT)
library(openxlsx)
library(data.table)

source("../Utils/Differential_Gene_Expression_Analysis.R")

# Protocol

This pipeline is based on the work of [Mariella Filbin](https://www.nature.com/articles/s41588-022-01236-3) and inspired of the code available [here](https://zenodo.org/records/7073167). Thanks a lot for the initialization of [Clara](omixanalytics@gmail.com)'s help for the DEG analyis.

# Load data

## scRNA-seq counts

In [ ]:
message("Import counts for frozen samples: ", params1$counts)
counts1 <- read.csv2(params1$counts, header = TRUE, sep = "\t")

message("Import counts fresh samples: ", params2$counts)
counts2 <- read.csv2(params2$counts, header = TRUE, sep = "\t")

counts.combined <- rbind(t(counts1), t(counts2))
dim(counts.combined)


## Metadata

In [ ]:
message("Import counts: ", params1$meta)
meta1 <- read.csv2(params1$meta, header = TRUE, sep = "\t")

message("Import counts: ", params2$meta)
meta2 <- read.csv2(params2$meta, header = TRUE, sep = "\t")

meta.combined <- rbind(meta1, meta2)
dim(meta.combined)


In [ ]:
meta3 <- read.csv(file = "../../scRNAseq_IlonLiu/scRNA_meta.csv")
sample_to_clinical_status <- setNames(meta3$clinical.status, meta3$Patient.ID)

# Map sample to clinical status and create new column
meta.combined <- meta.combined %>%
    mutate(clinical_status = sample_to_clinical_status[sample])
    

In [ ]:
meta.combined

In [ ]:
meta.combined <- meta.combined %>%
  mutate(annotation = ifelse(grepl("^OPC-like", annotation), "OPC-like", annotation))
  
meta.combined

## Data description

In [ ]:
table(meta.combined$location)
table(meta.combined$sample, meta.combined$location)
table(meta.combined)


## Import Seurat object

In [ ]:
dim(counts.combined)

In [ ]:
head(meta.combined)

In [ ]:
seu <- CreateSeuratObject(
  counts       = t(counts.combined),
  assay        = "RNA",
  meta.data    = meta.combined,
  project      = params$out_name,
  min.cells    = 0,
  min.features = 0
  #min.genes   = 0,
  #names.field = 1, 
  #names.delim = 1,
)


In [ ]:
seu

# Quality control

In [ ]:
DefaultAssay(seu) <- "RNA"

# Compute, for each cell, the proportion of reads in mitochondrial genes, and add to the metadata
pattern <- switch(params$species,
                 "h_sapiens" = "^MT-",
                 "m_musculus" = "^mt-"
                 )
seu[["percent.mito"]] <- PercentageFeatureSet(object = seu, pattern = pattern)

In [ ]:
# Compute, for each cell, the proportion of reads in ribosomal genes, and add to the metadata
pattern = switch(params$species,
                 "h_sapiens" = "^RPS|^RPL|^MRPS|^MRPL", # "^RP[SL]|^M?RP[SL]"
                 "m_musculus" = "^Rps|^Rpl|^Mrps|^Mrpl" # "^Rp[sl]|^M?rp[sl]"
                 )
seu[["percent.ribo"]] = PercentageFeatureSet(object = seu, pattern = pattern)

In [ ]:
# Add number of genes per UMI for each cell to metadata object
seu[["log10nGene"]] = log10(seu@meta.data$nFeature_RNA)

# Add number of genes per UMI for each cell to metadata object
seu[["log10nUMI"]] = log10(seu@meta.data$nCount_RNA)

# Add number of genes per UMI for each cell to metadata object
seu[["log10GenesPerUMI"]] = seu@meta.data$log10nGene/seu@meta.data$log10nUMI

# Normalization

## Log normalization & scaling

In [ ]:
# Normalize data
seu <- NormalizeData(
    object               = seu,
    normalization.method = "LogNormalize",
    scale.factor         = 1E4
)
    
# Detection of variable genes across the single cells
seu <- FindVariableFeatures(
    object           = seu,
    selection.method = "vst",
    nfeatures        = 2000
)

# Scaling the data and removing unwanted sources of variation
all.genes <- rownames(seu)
seu <- ScaleData(
  object    = seu,
  features  = all.genes,
  do.scale  = FALSE,
  do.center = TRUE
  #vars.to.regress = c("nUMI")
)


# Dimension reduction

In [ ]:
DefaultAssay(seu) <- "RNA"
# PCA
nPCs <- 50
seu <- RunPCA(
  object   = seu, 
  features = VariableFeatures(object = seu), 
  npcs     = nPCs
)
print(seu[["pca"]], nDims = 1:5, nFeatures = 1:5)

In [ ]:
VizDimLoadings(seu, dims = 1:2, reduction = "pca")

In [ ]:
DimHeatmap(seu, dims = 1, cells = 500, balanced = TRUE)

In [ ]:
DimHeatmap(seu, dims = 1:15, cells = 500, balanced = TRUE)

In [ ]:
ElbowPlot(object = seu, ndims = nPCs)
pcs <- elbow_pcs(seu, ndims = 50, graph = "pca")

# Harmony data integration 

In [ ]:
# Run Harmony
seu <- RunHarmony(
  seu,
  "sample",
  theta            = 2, 
  max.iter.harmony = 50,
  plot_convergence = TRUE
)

# Simple dim and vlnplot to examine integration 
DimPlot(object = seu, reduction = "harmony", pt.size = .1, group.by = "sample")

In [ ]:
DimHeatmap(seu, dims = 1, cells = 500, balanced = TRUE, reduction = "harmony")


In [ ]:
DimHeatmap(seu, dims = 1:15, cells = 500, balanced = TRUE, reduction = "harmony")


# UMAP and Clustering

In [ ]:
pcs <- elbow_pcs(seu, ndims = 50, graph = "harmony")


In [ ]:
resolution <- 0.6
reduction_type <-  "harmony"
dims <-  20
system.time({
seu %<>% RunUMAP(reduction = reduction_type, dims = 1:dims)
seu %<>% FindNeighbors(
  reduction    = reduction_type,
  dims         = 1:dims,
  force.recalc = TRUE
)    
seu %<>% FindClusters(resolution = resolution)
})
table(Idents(seu))

# UMAP visualization

In [ ]:
DimPlot(
  object     = seu, 
  group.by   = c("sample", "location", "annotation"), 
  label      = TRUE, 
  pt.size    = 3, 
  label.size = 0
) & NoAxes()

In [ ]:
DimPlot(seu, group.by = c("seurat_clusters", "RNA_snn_res.0.6", "annotation"), ncol = 3, label = TRUE, repel = TRUE)


In [ ]:
DimPlot(seu, group.by = "seurat_clusters", split.by = "location", label = TRUE, repel = TRUE)


In [ ]:
DimPlot(seu, group.by = "seurat_clusters", split.by = "annotation", label = TRUE, repel = TRUE)


In [ ]:
DimPlot(seu, group.by = "annotation", split.by = "location", label = TRUE, repel = TRUE)


# Cluster biomarkers

## SC analysis

### Between clusters

In [ ]:
seu

In [ ]:
cl_marker.sc.de <- FindAllMarkers(
  seu,
  only.pos        = TRUE, 
  logfc.threshold = 0.7,
  min.pct         = 0.5,
  test.use = "MAST"
)

cl_marker.sc.de <- cl_marker.sc.de[cl_marker.sc.de$p_val_adj <= 0.05, ]


In [ ]:
num_marker_genes <- 25
cl_marker.sc.de %>% group_by(cluster) %>% top_n(num_marker_genes, avg_log2FC) -> top_marker_genes
top_marker_genes <- data.frame(top_marker_genes)
datatable(top_marker_genes)


In [ ]:
head(top_marker_genes)

### Between cell types

In [ ]:
Idents(seu) <- seu@meta.data$annotation
cell_mark.sc.de <- FindAllMarkers(
  seu,
  only.pos        = TRUE, 
  logfc.threshold = 0.7,
  min.pct         = 0.5,
  test.use = "MAST"
)


In [ ]:
num_marker_genes <- 25
cell_mark.sc.de %>% group_by(cluster) %>% top_n(num_marker_genes, avg_log2FC) -> top_marker_genes
top_marker_genes <- data.frame(top_marker_genes)
datatable(top_marker_genes)
top_marker_genes

## Pseudobulking

### DE Location

In [ ]:
# pseudobulk the counts based on location-sample-celltype
pseudo_seu <- AggregateExpression(seu, assays = "RNA", return.seurat = T, group.by = c("sample", "annotation"))

# each 'cell' is a sample-celltype pseudobulk profile
tail(Cells(pseudo_seu))

Idents(pseudo_seu) <- seu@meta.data$annotation

bulk.de <- FindAllMarkers(  
object = pseudo_seu,
only.pos = TRUE, 
logfc.threshold = 0.7,
min.pct = 0.5,
test.use = "MAST")

In [ ]:
num_marker_genes <- 25
bulk.de %>% group_by(cluster) %>% top_n(num_marker_genes, avg_log2FC) -> top_marker_genes
top_marker_genes <- data.frame(top_marker_genes)
datatable(top_marker_genes)
top_marker_genes


In [ ]:
head(top_marker_genes)

In [ ]:
# PseudoBulking 
bulk.de.sg <- bulk.de[bulk.de$p_val_adj <= 0.05, ]

# scRNA
cell_mark.sc.de.sg <- cell_mark.sc.de[cell_mark.sc.de$p_val_adj <= 0.05, ]

# compare the DE P-values between the single-cell level and the pseudobulk level results
names(bulk.de.sg) <- paste0(names(bulk.de.sg), ".bulk")
bulk.de.sg$gene <- rownames(bulk.de.sg)

names(cell_mark.sc.de.sg) <- paste0(names(cell_mark.sc.de.sg), ".sc")
cell_mark.sc.de.sg$gene <- rownames(cell_mark.sc.de.sg)

merge_dat <- merge(cell_mark.sc.de.sg, bulk.de.sg, by = "gene")
merge_dat <- merge_dat[order(merge_dat$p_val.bulk), ]

# Number of genes that are marginally significant in both; marginally significant only in bulk; and marginally significant only in single-cell
common <- merge_dat$gene[which(merge_dat$p_val.bulk <= 0.05 & 
                                merge_dat$p_val.sc <= 0.05)]
                                
print(paste0('# scRNA DEG: ', length(unique(cell_mark.sc.de.sg$gene))))
print(paste0('# Pseudo-bulking DEG: ', length(unique(bulk.de.sg$gene))))
print(paste0('# Common DEG: ', length(common)))


# Export data

## Seurat object

In [ ]:
saveRDS(seu, file = here("output", glue("seu_", params$out_name, ".rds")))
saveRDS(pseudo_seu, file = here("output", glue("pseudo_seu_", params$out_name, ".rds")))


In [ ]:
# Cluster biomarkers
write.table(cl_marker.sc.de, file = here("output", glue("cl_biomarkers_", params$out_name, ".csv")), sep = ",", dec = ".", col.names = TRUE, row.names = FALSE)
write.xlsx(cl_marker.sc.de, file = here("output", glue("cl_biomarkers_", params$out_name, ".xlsx")))

# Cell biomarkers
write.table(cell_mark.sc.de.sg, file = here("output", glue("cell_biomarkers_", params$out_name, ".csv")), sep = ",", dec = ".", col.names = TRUE, row.names = FALSE)
write.xlsx(cell_mark.sc.de.sg, file = here("output", glue("cell_biomarkers_", params$out_name, ".xlsx")))

# Pseudobulking biomarkers
write.table(bulk.de.sg, file = here("output", glue("psebulk_", params$out_name, ".csv")), sep = ",", dec = ".", col.names = TRUE, row.names = FALSE)
write.xlsx(bulk.de.sg, file = here("output", glue("psebulk_", params$out_name, ".xlsx")))
